In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from HUGS.Interface import get_map_locations, parse_data, scatter_plot, comparison_plot
from HUGS.Interface import generate_password, get_download_keys, get_login, create_user
from HUGS.Interface import search_emissions, get_emissions_data, plot_emissions

from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, LogScale, Axis, Lines, Figure, Scatter

from ipywidgets import (GridspecLayout, GridBox , VBox, HBox, HTML, Layout, Text, 
                        Button, Output, Checkbox, Label)
import ipywidgets as widget

In [ ]:
%load_ext autoreload
%autoreload 2

# Welcome

This notebook allows interaction with the prototype of the HUS platform. It is currently very heavily in development and as such will change extensively in the coming months. A polished notebook interface based on [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) will be developed along with another, web-based, interface for users may not be as comfortable with Jupyer Notebooks.

Play around with the platform and please send us your feedback to [feedback@hugs-cloud.com](mailto:feedback@hugs-cloud.com)

 Enable matplotlib inline plotting

In [ ]:
%matplotlib notebook

# Remove this

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

# Create an account

To use the HUGS service an account must be created using the Acquire authentication service. Please follow the instructions below to create your account.

In [ ]:
create_user()

## Login

You are now ready to login to the HUGS platform

In [ ]:
user, login = get_login()
login

#### Check that we've been logged in

In [ ]:
user.wait_for_login()

# Searching

### To search

1. `search_terms` - a `list` of species to search for. To search for all species or locations simply pass an empty list.
  

2. `search_locations` - a `list` of locations / sites to search for. 
For example `bsd` or `Bilsdale`.
    
3. `data_type` - For example `CRDS` or `GC`
4. `start`, `end` - Python datetime objects 



In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
search = Search(service_url=base_url)

# A list of species names
search_terms = []
# A list of locations
search_locations = []
data_type = "CRDS"

start = datetime(1970, 1,1)
end = datetime.now()

search_results = search.search(search_terms=search_terms, locations=search_locations, 
                               data_type=data_type, start_datetime=start, end_datetime=end)


In [ ]:
search_results.keys()

### Map of locations

Here we utilise the [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/) library to create a map marked with the locations of the sites available. You may click on the marked locations to see details about the data held for these sites.

In [ ]:
get_map_locations(search_results)

### Playing with the data

In [ ]:
to_download = ["bsd_co", "hfd_co", "bsd_co2", "hfd_co2"]

Create a Retrieve object so we can interact with the data stored in the cloud.

In [ ]:
retrieve = Retrieve(service_url=base_url)

From the search results we can select the keys for the data we want to work with using the `get_download_keys` function. We just pass it a list containing the keys we want to download and it returns an object we can give to the `retrieve` function.

In [ ]:
download_keys = get_download_keys(search_results, to_download)
data = parse_data(retrieve.retrieve(keys=download_keys))

In [ ]:
bsd_data = data["bsd_co"]
scatter_plot(bsd_data)

In [ ]:
comparison_plot(data, ["hfd_co", "hfd_co2"])


## Plotting using matplotlib

The quick plots created above were created using [bqplot](https://github.com/bloomberg/bqplot/). You are free to use the plotting library of your choice to plot the data provided by the platform. For time-series data the data is provided as `Pandas.DataFrame` objects. Here we'll use `matplotlib` to make a few small plots.


We'll now plot multiple gases and place these plots into a single figure

In [ ]:
# A list of species names
search_terms = ["H-2402","CH3Cl","CH2Cl2","HFC-152a"]
# A list of locations
search_locations = ["capegrim"]
data_type = "GC"

start = datetime(1970, 1,1)
end = datetime.now()

search_results = search.search(search_terms=search_terms, locations=search_locations, 
                               data_type=data_type, start_datetime=start, end_datetime=end)

In [ ]:
search_results.keys()

In [ ]:
to_download = ["capegrim_H-2402", "capegrim_CH3Cl", "capegrim_CH2Cl2", "capegrim_HFC-152a"]
download_keys = get_download_keys(search_results, to_download)
data = parse_data(retrieve.retrieve(keys=download_keys))



In [ ]:
H2402_data = data["capegrim_H-2402"]["H-2402"]
CH3Cl_data = data["capegrim_CH3Cl"]["CH3Cl"]
CH2Cl2_data = data["capegrim_CH2Cl2"]["CH2Cl2"]
HFC152a_data = data["capegrim_HFC-152a"]["HFC-152a"]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

fig = plt.figure(tight_layout=True)
# Remove the spines
mpl.style.use('default')
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
# Get a nicer colourmap
cmap = plt.get_cmap("Set2")

ax = fig.add_subplot(221)
ax.plot(H2402_data.index, H2402_data.values, color=cmap(1), )
ax.set_title("H-2402")
ax.set_xticks(ax.get_xticks()[::4])
ax = fig.add_subplot(222)
ax.plot(CH3Cl_data.index, CH3Cl_data.values, color=cmap(2))
ax.set_title("CH3Cl")
ax.set_xticks(ax.get_xticks()[::4])
ax = fig.add_subplot(223)
ax.plot(CH2Cl2_data.index, CH2Cl2_data.values, color=cmap(3))
ax.set_title("CH2Cl2")
ax.set_xticks(ax.get_xticks()[::4])
ax = fig.add_subplot(224)
ax.plot(HFC152a_data.index, HFC152a_data.values, color=cmap(4))
ax.set_title("HFC152a")
ax.set_xticks(ax.get_xticks()[::4])

## Plot emissions maps

In [ ]:
# Empty search terms, show all data
search_terms = []
results = search_emissions(search_terms=search_terms)
print(results)

In [ ]:
data = get_emissions_data("WAO-20magl_EUROPE_201501")

plot_emissions(data)